<a href="https://colab.research.google.com/github/hmu-mech-npap/diss.tn.filtering-wind.py/blob/main/src/Torosian_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import math
import matplotlib.pyplot as plt 
from scipy import signal

In [ ]:
# N-order TH ORDER BUTTERWORTH FILTER WITH A GAIN DROP OF 1/sqrt(2) AT 0.4 CYCLES/SAMPLE
n_order=2
bb, ab  = signal.butter ( n_order, 0.5, 'low', analog=False, output='ba')
print ('Coefficients of b = ', bb)
print ('Coefficients of a = ', ab)
wb, hb = signal.freqz(bb, ab)
wb = wb/(2*math.pi)


plt.plot(wb, abs(np.array(hb)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [cycles/sample]')
plt.ylabel('Amplitute [dB]')
plt.xscale('log')
plt.yscale('log')
plt.margins(0, 0.1)
plt.grid(which = 'both', axis='both')
plt.savefig('Butterworth Filter Freq Response.png')


In [ ]:
# 4TH ORDER BESSEL FILTER WITH A GAIN DROP OF 1/sqrt(2) AT 0.4 CYCLES/SAMPLE

bb, ab = signal.bessel (4, 0.8, 'low', analog=False, output='ba')
print ('Coefficients of b = ', bb)
print ('Coefficients of a = ', ab)
wb, hb = signal.freqz(bb, ab)
wb = wb/(2*math.pi)
plt.plot(wb, abs(np.array(hb)))

plt.title('Bessel filter frequency response')
plt.xlabel('Frequency [cycles/sample]')
plt.ylabel('Amplitute [dB]')
plt.margins(0, 0.1)
plt.grid(which= 'both', axis= 'both')
plt.savefig('Bessel Filter Freq Response.png')

In [ ]:
#4TH ORDER CHEBYSHEV FILTER TYPE 1 (ONLY IN PASSBAND RIPPLES) WITH MAX RIPPLES=2 AND THE GAIN DROP AT 1.5 CYCLES/SAMPLE

bb, ab = signal.cheby1 (4, 2, 0.3, 'low', analog=False, output='ba')
print ('Coefficients of b = ', bb)
print ('Coefficients of a = ', ab)
wb, hb = signal.freqz(bb, ab)
wb = wb/(2*math.pi)
plt.plot(wb, abs(np.array(hb)))

plt.title('Chebyshev filter frequency response')
plt.xlabel('Frequency [cycles/sample]')
plt.ylabel('Amplitute [dB]')
plt.margins(0, 0.1)
plt.grid(which= 'both', axis= 'both')
plt.savefig('Chebyshev Filter Freq Response.png')

In [ ]:
# 4TH ORDER ELLIPTIC FILTER WITH MAX RIPPLES =2dB IN PASSBAND, MIN ATTENUATION =8dB IN STOP BAND AT 0.25 CYCLES/SAMPLE

bb, ab = signal.ellip (4, 2, 8, 0.5, 'low', analog=False, output='ba')
print ('Coefficients of b = ', bb)
print ('Coefficients of a = ', ab)
wb, hb = signal.freqz(bb, ab)
wb = wb/(2*math.pi)
plt.plot(wb, abs(np.array(hb)))

plt.title('Elliptic filter frequency response')
plt.xlabel('Frequency [cycles/sample]')
plt.ylabel('Amplitute [dB]')
plt.margins(0, 0.1)
plt.grid(which= 'both', axis= 'both')
plt.savefig('Elliptic Filter Freq Response.png')

# applying a filter

In [ ]:
def generate_random_sig():
  t = np.linspace(0, 1, 1000, False)  # 1 second
  sig = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t) + np.random.rand(t.shape[0])
  return (t,sig)
t, sig = generate_random_sig()



In [ ]:
sos = signal.butter(10, 30, 'lp', fs=1000, output='sos')
filtered = signal.sosfilt(sos, sig)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
fig.suptitle('Filtering of signal with f1=10[Hz], f2=20[Hz] and noise')
ax1.plot(t, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids + noise')
ax1.axis([0, 1, -2, 2])
ax2.plot(t, filtered)
ax2.set_title('After filter')
ax2.axis([0, 1, -2, 2])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()
plt.show()

# Power spectrum

In [ ]:
# signal.welch
fs=1000
f, Pxx_spec = signal.welch(sig, fs, 'flattop', 1024, scaling='spectrum')
plt.figure()
plt.semilogy(f, np.sqrt(Pxx_spec))
plt.xlabel('frequency [Hz]')
plt.ylabel('Linear spectrum [V RMS]')
plt.title('Power spectrum (scipy.signal.welch)')
plt.show()

In [ ]:
# signal.welch
fs=1000
f, Pxx_spec = signal.welch(filtered, fs, 'flattop', 1024, scaling='spectrum')
plt.figure()
plt.semilogy(f, np.sqrt(Pxx_spec))
plt.xlabel('frequency [Hz]')
plt.ylabel('Linear spectrum [V RMS]')
plt.title('Power spectrum (scipy.signal.welch)')
plt.show()

# Understanding butter filter 
the parameter used for the cutoff frequency can be ambiguous. 

I.e. the power spectrum should have value until half of the sampling frequnecy (fs/2), however some plots are normalised with respect to the sampling frequency.

The following graphs put in a single log-log plot the:
- raw signal
- filtered signal
- filter response for comparison.

In [ ]:
def generate_random_sig():
  t = np.linspace(0, 1, 1000, False)  # 1 second
  sig = np.sin(2*np.pi*10*t) + np.random.rand(t.shape[0])   #+ np.sin(2*np.pi*20*t) 
  sig = np.sin(2*np.pi*10*t) + np.random.rand(t.shape[0])  + np.sin(2*np.pi*20*t) 
    
  return (t,sig)
t, sig = generate_random_sig()


__Filter options:__
- N : int
    The order of the filter.
- Wn : array_like
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 sequence.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the "-3 dB point").
    For digital filters, Wn are in the same units as fs. By default, fs is 2 half-cycles/sample, so these are normalized from 0 to 1, where 1 is the Nyquist frequency. 
    (Wn is thus in half-cycles / sample.)

    For analog filters, Wn is an angular frequency (e.g. rad/s).
- btype : {'lowpass', 'highpass', 'bandpass', 'bandstop'}, optional
    The type of filter. Default is 'lowpass'.
- analog : bool, optional
    When True, return an analog filter, otherwise a digital filter is returned.
- output : {'ba', 'zpk', 'sos'}, optional
    Type of output: numerator/denominator ('ba'), pole-zero ('zpk'), or second-order sections ('sos'). Default is 'ba' for backwards compatibility, but 'sos' should be used for general-purpose filtering.
- fs : float, optional
    The sampling frequency of the digital system.


In [ ]:
n_order = 5
Wn = 10 # cutoff frequency
fs_hz=1000
sos = signal.butter(N= n_order, Wn= Wn, btype= 'lp', fs=fs_hz, output='sos')
filtered = signal.sosfilt(sos, sig)

# calculate spectrum 
f, Pxx_spec = signal.welch(sig, fs_hz, 'flattop', 1024, scaling='spectrum')
f, Pxx_spec_filt = signal.welch(filtered, fs_hz, 'flattop', 1024, scaling='spectrum')

wb, hb = signal.sosfreqz(sos)
fb = wb/(2*math.pi)

In [ ]:

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
fig.suptitle('Time Domain Filtering of signal with f1=10[Hz], f2=20[Hz] and noise')
ax1.plot(t, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids + noise')
ax1.axis([0, 1, -2, 2])
ax2.plot(t, filtered)
ax2.set_title('After filter')
ax2.axis([0, 1, -2, 2])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()

In [ ]:
%matplotlib inline
# %matplotlib qt
plt.plot(fb, 2e-2*abs(np.array(hb)))
plt.semilogy(f/fs_hz, np.sqrt(Pxx_spec))
plt.semilogy(f/fs_hz, np.sqrt(Pxx_spec_filt))
plt.title('Butter filter frequency response')
plt.xlabel('Frequency [cycles/sample]')
plt.ylabel('Amplitute [dB]')
plt.xscale('log')
plt.yscale('log')
plt.margins(0, 0.1)
plt.grid(which= 'both', axis= 'both')
plt.ylim([1e-7, 1])
# plt.savefig('Bessel Filter Freq Response.png')

In [ ]:
fc_hz = 50
sos = signal.butter(N= n_order, Wn=fc_hz, btype= 'lp', fs=fs_hz, output='sos')
def filt_func(sig, fs_hz):
    
    filtered = signal.sosfilt(sos, sig)    
    return filtered

def plot_comparative_response(sig, # cutoff frequency
        filter_func,fs_hz=1000):
    
    filtered = filter_func(sig, fs_hz)
    
    # calculate spectrum 
    f, Pxx_spec = signal.welch(sig, fs_hz, 'flattop', 1024, scaling='spectrum')
    f, Pxx_spec_filt = signal.welch(filtered, fs_hz, 'flattop', 1024, scaling='spectrum')

    wb, hb = signal.sosfreqz(sos)
    fb = wb/(2*math.pi)
    
    # plot time domain 
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    fig.suptitle('Time Domain Filtering of signal with f1=10[Hz], f2=20[Hz] and noise')
    ax1.plot(t, sig)
    ax1.set_title('10 Hz and 20 Hz sinusoids + noise')
    ax1.axis([0, 1, -2, 2])
    ax2.plot(t, filtered)
    ax2.set_title('After filter')
    ax2.axis([0, 1, -2, 2])
    ax2.set_xlabel('Time [seconds]')
    plt.tight_layout()
    
    fig, ax1 = plt.subplots(1, 1, sharex=True)
    plt.plot(fb, 2e-2*abs(np.array(hb)), label='raw ')
    plt.semilogy(f/fs_hz, np.sqrt(Pxx_spec), label='filtered')
    plt.semilogy(f/fs_hz, np.sqrt(Pxx_spec_filt), label='response ')
    plt.title('Butter filter frequency response')
    plt.xlabel('Frequency [cycles/sample]')
    plt.ylabel('Amplitute [dB]')
    plt.xscale('log')
    plt.yscale('log')
    plt.margins(0, 0.1)
    plt.grid(which= 'both', axis= 'both')
    plt.ylim([1e-7, 1])
    plt.legend()
    # plt.savefig('Bessel Filter Freq Response.png')
    
plot_comparative_response(sig, # cutoff frequency
        filter_func=filt_func,fs_hz=1000)
    